#### Import Dependencies

In [2]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PayloadSchemaType, PointStruct, SparseVectorParams, Document, Prefetch, FusionQuery
from qdrant_client import models

import pandas as pd
import openai
import cohere

c:\Users\dushy\Desktop\globe\study\ai\swirl25oct\Project1\ai-engineering-bootcamp-oct25\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Retrieval

In [3]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [4]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=[text],
        model=model,
    )
    return response.data[0].embedding

In [5]:
def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-01-hybrid-search",
        prefetch=[
            Prefetch(
                query=query_embedding,
                using="text-embedding-3-small",
                limit=20
            ),
            Prefetch(
                query=Document(
                    text=query,
                    model="qdrant/bm25"
                ),
                using="bm25",
                limit=20
            )
        ],
        query=FusionQuery(fusion="rrf"),
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    retrieved_context_ratings = []
    similarity_scores = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }

In [6]:
query = "Can I get some tablet?"

In [7]:

results = retrieve_data(query, qdrant_client, k=20)

In [8]:
results

{'retrieved_context_ids': ['B0B8NVNQKX',
  'B09LH466KZ',
  'B0BR33XH8D',
  'B0BN7WWH63',
  'B0C78B1BTB',
  'B0BL2CZSHT',
  'B0BFGXRMJN',
  'B08CGM6Y1J',
  'B0C3XYD574',
  'B0C772GZRF',
  'B0BTPG4XHH',
  'B0BVZ47N6N',
  'B0BC78N85F',
  'B0BRXZDBXZ',
  'B0BMVR2MVC',
  'B0BBPVCFW7',
  'B0BP1YXF3R',
  'B0BS15TRJ3',
  'B09PYKX76X',
  'B0BSGDCVND'],
 'retrieved_context': ['COOPERS 7 inch Kids Tablet Android 11 Tablet for Kids, 2GB RAM + 32GB ROM Toddler Tablet PC for Children, IPS Touch Screen, Dual Camera, Dual Speaker, WiFi Computer Tablet, Light Blue ✿【Good Kids tablet】This 7 inch tablet for kids with silm body and lightweight, it is easy to hold by children. Also the special design can protect the tablet well when dropping. ✿【Parental Control】Toddlers Tablet with parent mode can add or block apps. Set screen time limits. This tablet come with iwawa app. kids can get access to fun and educational games and videos. ✿【Powerful Tablets】Equipmented with quad core CPU, Android 11.0 System, 32G

#### Reranking

In [9]:
cohere_client = cohere.ClientV2()

In [10]:
to_rerank = results["retrieved_context"]

In [11]:
response = cohere_client.rerank(
    model="rerank-v3.5",
    query=query,
    documents=to_rerank,
    top_n=20
)

In [12]:
response

V2RerankResponse(id='6d970a18-abc9-40a8-ab1d-68d5616c1d68', results=[V2RerankResponseResultsItem(index=19, relevance_score=0.21011764), V2RerankResponseResultsItem(index=0, relevance_score=0.17562322), V2RerankResponseResultsItem(index=8, relevance_score=0.16636397), V2RerankResponseResultsItem(index=4, relevance_score=0.16547202), V2RerankResponseResultsItem(index=1, relevance_score=0.1605162), V2RerankResponseResultsItem(index=5, relevance_score=0.1513414), V2RerankResponseResultsItem(index=13, relevance_score=0.15119094), V2RerankResponseResultsItem(index=14, relevance_score=0.14032473), V2RerankResponseResultsItem(index=9, relevance_score=0.13983068), V2RerankResponseResultsItem(index=11, relevance_score=0.13947868), V2RerankResponseResultsItem(index=3, relevance_score=0.13849701), V2RerankResponseResultsItem(index=6, relevance_score=0.13165279), V2RerankResponseResultsItem(index=12, relevance_score=0.12324974), V2RerankResponseResultsItem(index=15, relevance_score=0.10670012), V2R

##### cohere runs a cross encoder model

In [13]:
reranked_results = [to_rerank[results.index] for results in response.results]

In [14]:
results["retrieved_context"]

['COOPERS 7 inch Kids Tablet Android 11 Tablet for Kids, 2GB RAM + 32GB ROM Toddler Tablet PC for Children, IPS Touch Screen, Dual Camera, Dual Speaker, WiFi Computer Tablet, Light Blue ✿【Good Kids tablet】This 7 inch tablet for kids with silm body and lightweight, it is easy to hold by children. Also the special design can protect the tablet well when dropping. ✿【Parental Control】Toddlers Tablet with parent mode can add or block apps. Set screen time limits. This tablet come with iwawa app. kids can get access to fun and educational games and videos. ✿【Powerful Tablets】Equipmented with quad core CPU, Android 11.0 System, 32GB Big Storage, 1024*600 IPS Screen offer a clear view. Runs Kinds of apps and game for kids smoothly. ✿【Long Lasting】Tablet for kids built with large capacity battery. For mixed use up to 6 hours. You can enjoy happy parent-child time with your kids. ✿【Best Gift】This high performance Children tablet is designed specially for kids. Best gift choice for Christmas, New

In [15]:
reranked_results

["Missnine Tote Bag Canvas Laptop Bag 15.6 inch Briefcase for Women Large Capacity Handbag for Office, School, Travel Canvas+Pu Leather Imported Polyester lining Zipper closure Large Capacity: The roomy structure can accommodate your work and life requirements. Spacious compartment with a padded laptop section for better protection of a 15.6'' computer and a tablet slot for a 10.9'' iPad, keep your tech product well-organized Multi-pockets: 8 pockets keeping all your items organized and within reach. Convenient exterior front pocket that is easy to access for your essentials. Main pocket with a zipper closure for extra safety, 1 interior zipper pocket, 2 open pockets and 2 pen slots for your necessities Durable & Sturdy: The water-resistant canvas fabric, decorated with PU leather, prevents it from getting wet on rainy days. 4 rivets on the bottom can effectively stabilize your items to protect the base from damage and against accidental impacts. Stylish & Practical: Carry the fashiona